In [1]:
import numpy as np
from scipy.optimize import differential_evolution
from differential_evolution import DifferentialEvolution
from genetic_algorithm import GeneticAlgorithm
from jade import JADifferentialEvolution
from problemas import PressureVessel, TensionCompressionSpring, SpeedReducer
import pandas as pd

In [2]:
def run(algorithm, problem, N, G, config):
    res = dict()
    for case in config:
        res[case] = []
        for i in range(N):
            alg = algorithm(problem.problem, problem.bounds, seed=i, G=G, **config[case])
            alg.run()
            res[case].append(alg.fun)
    return res

In [3]:
class ScipyAdapter:
    def __init__(self, problem, bounds, seed, G, mutation, recombination):
        self.func = differential_evolution
        self.result = None
        self.problem = problem
        self.bounds = bounds
        self.seed = seed
        self.maxiter = G
        self.mutation = mutation
        self.recombination = recombination

    @property
    def fun(self):
        return self.result.fun
        
    def run(self):
        self.result = self.func(self.problem, bounds=self.bounds, seed=self.seed, maxiter=self.maxiter, mutation=self.mutation, recombination=self.recombination)


In [4]:
N = 30
G = 200

In [5]:
config = {'case1':{'mutation':0.7, 'recombination':0.7},
          'case2':{'mutation':0.6, 'recombination':0.8},
          'case3':{'mutation':0.8, 'recombination':0.6}}

# 4.3.1 Example 8: Design of Pressure Vessel

In [6]:
best_x = (0.8125, 0.4375, 42.098446, 176.636596)
pv = PressureVessel()
pv.pressure_vessel(best_x)-pv.constrains(best_x) # = 6059.714339

6059.714406596527

In [7]:
res_de = run(algorithm=DifferentialEvolution, problem=pv, N=N, G=G, config=config)
res_ga = run(algorithm=GeneticAlgorithm, problem=pv, N=N, G=G, config=config)
res_jade = run(algorithm=JADifferentialEvolution, problem=pv, N=N, G=G, config=config)
res_scipy = run(algorithm=ScipyAdapter, problem=pv, N=N, G=G, config=config)

In [8]:
df_scipy = pd.DataFrame(res_scipy)
df_jade = pd.DataFrame(res_jade)
df_de = pd.DataFrame(res_de)
df_ga = pd.DataFrame(res_ga)

print('Scipy DE')
print(df_scipy.describe())
print()
print('DE')
print(df_de.describe())
print()
print('GA')
print(df_ga.describe())
print()
print('JADE')
print(df_jade.describe())

Scipy DE
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   6129.063891  6344.126139  6092.374045
std     123.364782   291.980889    82.679927
min    6059.932671  6059.733873  6059.731881
25%    6061.250665  6091.043242  6061.021478
50%    6091.151410  6371.054639  6063.480757
75%    6093.265213  6413.496441  6091.707510
max    6419.882870  6824.338635  6411.791999

DE
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   6061.783193  6059.729713  6081.522931
std       2.771037     0.020929    15.729263
min    6059.899275  6059.714788  6060.252735
25%    6060.231963  6059.717626  6071.956076
50%    6061.043693  6059.722702  6077.537347
75%    6062.335561  6059.730658  6085.606400
max    6074.836493  6059.807144  6119.812004

GA
               case1          case2          case3
count      30.000000      30.000000      30.000000
mean   216178.697828  216178.697828  216178.697828
std     99343.516968

# 4.3.3 Example 10: Design of Tension/Compression Spring

In [9]:
best_x = (0.051749, 0.358179, 11.203763)
tcs = TensionCompressionSpring()
tcs.tension_compression_spring(best_x)-tcs.constrains(best_x)

0.012664899963644788

In [10]:
res_de = run(algorithm=DifferentialEvolution, problem=tcs, N=N, G=G, config=config)
res_ga = run(algorithm=GeneticAlgorithm, problem=tcs, N=N, G=G, config=config)
res_jade = run(algorithm=JADifferentialEvolution, problem=tcs, N=N, G=G, config=config)
res_scipy = run(algorithm=ScipyAdapter, problem=tcs, N=N, G=G, config=config)

In [11]:
df_scipy = pd.DataFrame(res_scipy)
df_jade = pd.DataFrame(res_jade)
df_de = pd.DataFrame(res_de)
df_ga = pd.DataFrame(res_ga)

print('Scipy DE')
print(df_scipy.describe())
print()
print('DE')
print(df_de.describe())
print()
print('GA')
print(df_ga.describe())
print()
print('JADE')
print(df_jade.describe())

Scipy DE
           case1      case2      case3
count  30.000000  30.000000  30.000000
mean    0.012710   0.012749   0.012697
std     0.000054   0.000097   0.000026
min     0.012668   0.012671   0.012667
25%     0.012678   0.012688   0.012680
50%     0.012690   0.012719   0.012690
75%     0.012726   0.012788   0.012704
max     0.012932   0.013174   0.012787

DE
           case1         case2      case3
count  30.000000  3.000000e+01  30.000000
mean    0.012669  1.266546e-02   0.012697
std     0.000004  2.287372e-07   0.000033
min     0.012666  1.266525e-02   0.012667
25%     0.012667  1.266529e-02   0.012676
50%     0.012667  1.266542e-02   0.012689
75%     0.012669  1.266553e-02   0.012699
max     0.012681  1.266634e-02   0.012831

GA
           case1      case2      case3
count  30.000000  30.000000  30.000000
mean    2.369622   2.369622   2.369622
std     3.388809   3.388809   3.388809
min     0.015261   0.015261   0.015261
25%     0.091149   0.091149   0.091149
50%     0.230142   0

# 4.3.4 Example 11: Design of a Speed Reducer

In [12]:
best_x = (3.4999, 0.69999, 17, 7.3, 7.8, 3.3502, 5.2866)
sr = SpeedReducer()
sr.speed_reducer(best_x)-sr.constrains(best_x)

2996.207037654751

In [13]:
res_de = run(algorithm=DifferentialEvolution, problem=sr, N=N, G=G, config=config)
res_ga = run(algorithm=GeneticAlgorithm, problem=sr, N=N, G=G, config=config)
res_jade = run(algorithm=JADifferentialEvolution, problem=sr, N=N, G=G, config=config)
res_scipy = run(algorithm=ScipyAdapter, problem=sr, N=N, G=G, config=config)

In [14]:
df_scipy = pd.DataFrame(res_scipy)
df_jade = pd.DataFrame(res_jade)
df_de = pd.DataFrame(res_de)
df_ga = pd.DataFrame(res_ga)

print('Scipy DE')
print(df_scipy.describe())
print()
print('DE')
print(df_de.describe())
print()
print('GA')
print(df_ga.describe())
print()
print('JADE')
print(df_jade.describe())

Scipy DE
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   3009.584330  3009.638277  3014.471702
std       4.286431     3.804290     5.026209
min    3001.662101  3003.393361  3007.273549
25%    3006.693041  3005.976877  3010.909947
50%    3009.937761  3010.521262  3012.291645
75%    3012.248191  3011.783684  3018.217274
max    3017.145742  3015.761840  3026.216066

DE
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   2996.348388  2996.348314  2996.348386
std       0.000173     0.000083     0.000145
min    2996.348206  2996.348200  2996.348199
25%    2996.348268  2996.348261  2996.348298
50%    2996.348363  2996.348306  2996.348337
75%    2996.348441  2996.348352  2996.348448
max    2996.349035  2996.348580  2996.348844

GA
             case1        case2        case3
count    30.000000    30.000000    30.000000
mean   3768.983599  3768.983599  3768.983599
std     886.384442   886.384442   886.